In [1]:
from owlready2 import *
print("Owlready2 is installed and ready to use!")

Owlready2 is installed and ready to use!


In [2]:
def analyze_ontology(ontology):
    print("\nAnalyzing ontology...")

    # Count instances
    instances = list(ontology.individuals())
    num_instances = len(instances)
    print(f"Number of instances: {num_instances}")

    # Iterate through each instance
    for instance in instances:
        print(f"\nInstance: {instance.name}")

        # Get all properties assigned to the instance
        properties = list(instance.get_properties())
        
        # Separate data properties and object properties
        data_properties = [prop for prop in properties if isinstance(prop, DataPropertyClass)]
        object_properties = [prop for prop in properties if isinstance(prop, ObjectPropertyClass)]

        # Data properties
        print(f"  Data properties assigned: {len(data_properties)}")
        for prop in data_properties:
            values = getattr(instance, prop.python_name, [])
            print(f"    - {prop.name}: {values}")

        # Object properties
        print(f"  Object properties assigned: {len(object_properties)}")
        for prop in object_properties:
            values = getattr(instance, prop.python_name, [])
            if values:
                related_names = [related.name for related in values]
                print(f"    - {prop.name}: {related_names}")
            else:
                print(f"    - {prop.name}: None")
            
def check_ontology(onto):
    print("\nChecking ontology consistency...")

    try:
        # Run the reasoner
        with onto:
            sync_reasoner_pellet() 

        # Check for inconsistent classes after instantiation of Circles use case
        inconsistent_classes = list(onto.inconsistent_classes())
        if inconsistent_classes:
            print("\nInconsistent classes found:")
            for cls in inconsistent_classes:
                # Print the class name or IRI
                print(f"Class: {cls.name}, IRI: {cls.iri}")
        else:
            print("\nThe ontology is consistent!")

    except OwlReadyInconsistentOntologyError as e:
        print("\nOntology is inconsistent. The reasoner reported an error:")
        print(e)

def load_ontology(ontology_path):
    # Load the ontology
    onto = get_ontology(ontology_path).load()

    # Display classes in the ontology
    print("Classes in the ontology:")
    for cls in onto.classes():
        print(cls)

    return onto

# Utility function to retrieve a class by name
def get_class_by_name(ontology, class_name):
        return next((cls for cls in ontology.classes() if cls.name == class_name), None)


def instantiate_ontology(onto):
    
    # Retrieve the DAO class
    DAOClass = get_class_by_name(onto, "DAO")
    if not DAOClass:
        raise ValueError("DAO class not found in the ontology!")

    # Create the CirclesDAO individual
    circles_dao = DAOClass("CirclesDAO")
    circles_dao.DAO_name.append("Circles DAO")
    circles_dao.mission_statement.append("Manage governance of Circles group, token economy, and attest user contributions.")

    # Define roles
    RoleClass = get_class_by_name(onto, "Role")
    if not RoleClass:
        raise ValueError("Role class not found in the ontology!")

    group_member = RoleClass("GroupMember")
    group_member.role_description.append("Users depositing Circles tokens to join.")

    active_member = RoleClass("ActiveMember")
    active_member.role_description.append("Members contributing to the group governance.")

    treasury_manager = RoleClass("TreasuryManager")
    treasury_manager.role_description.append("Manages group funds.")

    # Define committees
    CommitteeClass = get_class_by_name(onto, "Committee")
    if not CommitteeClass:
        raise ValueError("Committee class not found in the ontology!")

    general_assembly = CommitteeClass("GeneralAssembly")
    general_assembly.n_agents.append(100)

    community_council = CommitteeClass("CommunityCouncil")
    community_council.n_agents.append(10)


    economic_council = CommitteeClass("EconomicCouncil")
    economic_council.n_agents.append(15)


    technical_council = CommitteeClass("TechnicalCouncil")
    technical_council.n_agents.append(7)


    # Define governance areas
    GovernanceAreaClass = get_class_by_name(onto, "Governance_Area")
    if not GovernanceAreaClass:
        raise ValueError("Governance_Area class not found in the ontology!")

    service_provision = GovernanceAreaClass("ServiceProvision")
    service_provision.description.append("Proposes and evaluates services for the Circles ecosystem.")
    service_provision.implementation.append("hybrid")

    treasury_management = GovernanceAreaClass("TreasuryManagement")
    treasury_management.description.append("Manages DAO funds.")
    treasury_management.implementation.append("onchain")

    group_membership = GovernanceAreaClass("GroupMembership")
    group_membership.description.append("Defines membership requirements.")
    group_membership.implementation.append("onchain")

    # Define tokens 
    TokenTypeClass = get_class_by_name(onto, "token_type")
    if not TokenTypeClass:
        raise ValueError("token_type class not found in the ontology!")

    circles_token = TokenTypeClass("CirclesToken")
    circles_token.token_name.append("Circles Token")
    circles_token.fungibility.append("SFT")
    circles_token.transferable.append(True)
    circles_token.token_utility.extend(["financial", "governance"])

    group_currency = TokenTypeClass("GroupCurrency")
    group_currency.token_name.append("Group Currency")
    group_currency.fungibility.append("FT")
    group_currency.transferable.append(True)
    group_currency.token_utility.extend(["financial", "access"])

    perishable_share = TokenTypeClass("PerishableShare")
    perishable_share.token_name.append("Perishable Share")
    perishable_share.fungibility.append("SFT")
    perishable_share.transferable.append(True)
    perishable_share.token_utility.extend(["reputation", "governance"])

    # Define voting protocols 
    DecisionMakingPolicyClass = get_class_by_name(onto, "decision_making_policy")
    if not DecisionMakingPolicyClass:
        raise ValueError("decision_making_policy class not found in the ontology!")

    voting_protocol_general = DecisionMakingPolicyClass("GeneralAssemblyVoting")
    voting_protocol_general.DM_method.append("Optimistic Governance")
    voting_protocol_general.voting_requirement.append("Group Currency ownership")
    voting_protocol_general.proposal_requirement.append("Group Currency ownership")

    voting_protocol_council = DecisionMakingPolicyClass("CouncilVoting")
    voting_protocol_council.DM_method.append("Relative Majority Voting")
    voting_protocol_council.voting_requirement.append("Perishable Share ownership")
    voting_protocol_council.proposal_requirement.append("Perishable Share ownership")

    # Assign roles and permissions 
    RoleAssignmentClass = get_class_by_name(onto, "Role_Assignment")
    if not RoleAssignmentClass:
        raise ValueError("Role_Assignment class not found in the ontology!")

    service_provision_role = RoleAssignmentClass("ServiceProvisionRole")
    service_provision_role.role_ID.append("GeneralAssemblyRole")
    service_provision_role.assignment_method.append("Group Currency ownership")

    membership_management = RoleAssignmentClass("MembershipManagement")
    membership_management.role_ID.append("CommunityCouncilRole")
    membership_management.assignment_method.append("Perishable Share ownership")
    
    # Save the ontology
    onto.save(file="Circles_Ubi_Instantiated.owl", format="rdfxml")
    print("Ontology instantiated and saved as 'Circles_Ubi_Instantiated.owl'.")
    
def reset_ontology(ontology):
    """Resets the ontology by clearing its contents from memory."""
    # Destroy all entities in the ontology
    for entity in list(ontology.classes()) + list(ontology.individuals()) + list(ontology.object_properties()) + list(ontology.data_properties()):
        destroy_entity(entity)

    # Remove the ontology from the default world
    if ontology.base_iri in default_world.ontologies:
        del default_world.ontologies[ontology.base_iri]

    # Return None to indicate the ontology has been cleared
    return None

In [3]:
ontology_path = "/home/sowelo/Scrivania/DAOMod/DAOMod/DAOMod Ontology/DAOMod_Ontology.rdf"
onto = load_ontology(ontology_path)
check_ontology(onto)
instantiate_ontology(onto)
check_ontology(onto)
analyze_ontology(onto)

* Owlready2 * Running Pellet...
    java -Xmx2000M -cp /home/sowelo/Scrivania/DAOMod/DAOMod/DAOMod Ontology/myenv/lib/python3.12/site-packages/owlready2/pellet/owlapi-distribution-3.4.3-bin.jar:/home/sowelo/Scrivania/DAOMod/DAOMod/DAOMod Ontology/myenv/lib/python3.12/site-packages/owlready2/pellet/log4j-core-2.19.0.jar:/home/sowelo/Scrivania/DAOMod/DAOMod/DAOMod Ontology/myenv/lib/python3.12/site-packages/owlready2/pellet/jena-iri-0.9.5.jar:/home/sowelo/Scrivania/DAOMod/DAOMod/DAOMod Ontology/myenv/lib/python3.12/site-packages/owlready2/pellet/jena-arq-2.10.0.jar:/home/sowelo/Scrivania/DAOMod/DAOMod/DAOMod Ontology/myenv/lib/python3.12/site-packages/owlready2/pellet/xml-apis-1.4.01.jar:/home/sowelo/Scrivania/DAOMod/DAOMod/DAOMod Ontology/myenv/lib/python3.12/site-packages/owlready2/pellet/commons-codec-1.6.jar:/home/sowelo/Scrivania/DAOMod/DAOMod/DAOMod Ontology/myenv/lib/python3.12/site-packages/owlready2/pellet/httpclient-4.2.3.jar:/home/sowelo/Scrivania/DAOMod/DAOMod/DAOMod Ontology

Classes in the ontology:
eSourcing_Ontology.Contract
DAOMod_Ontology.Actor
eSourcing_Ontology.Reaction_Rule
eSourcing_Ontology.Copier_Action
eSourcing_Ontology.Enabler_Action
eSourcing_Ontology.Executive_Action
eSourcing_Ontology.Link_Property
eSourcing_Ontology.Sink
eSourcing_Ontology.Passing_Type
eSourcing_Ontology.Block
eSourcing_Ontology.Aim_Case
eSourcing_Ontology.Scope
eSourcing_Ontology.Sourcing_Sphere
eSourcing_Ontology.Aim_Task
eSourcing_Ontology.Document
eSourcing_Ontology.Var_Section
eSourcing_Ontology.Privilege
eSourcing_Ontology.State_Constraint_Rule
eSourcing_Ontology.Orgnizational_Position
eSourcing_Ontology.Appointment
eSourcing_Ontology.Task
eSourcing_Ontology.Role
eSourcing_Ontology.Mediator
eSourcing_Ontology.Party
eSourcing_Ontology.Rule_Section
eSourcing_Ontology.Snippet_Section
eSourcing_Ontology.Process
eSourcing_Ontology.Process_Section
eSourcing_Ontology.Role_Delegation
eSourcing_Ontology.Cancelled
eSourcing_Ontology.Terminated
eSourcing_Ontology.Unfulfilled
eS

* Owlready2 * Pellet took 0.899362325668335 seconds
* Owlready * (NB: only changes on entities loaded in Python are shown, other changes are done but not listed)
* Owlready2 * Running Pellet...
    java -Xmx2000M -cp /home/sowelo/Scrivania/DAOMod/DAOMod/DAOMod Ontology/myenv/lib/python3.12/site-packages/owlready2/pellet/owlapi-distribution-3.4.3-bin.jar:/home/sowelo/Scrivania/DAOMod/DAOMod/DAOMod Ontology/myenv/lib/python3.12/site-packages/owlready2/pellet/log4j-core-2.19.0.jar:/home/sowelo/Scrivania/DAOMod/DAOMod/DAOMod Ontology/myenv/lib/python3.12/site-packages/owlready2/pellet/jena-iri-0.9.5.jar:/home/sowelo/Scrivania/DAOMod/DAOMod/DAOMod Ontology/myenv/lib/python3.12/site-packages/owlready2/pellet/jena-arq-2.10.0.jar:/home/sowelo/Scrivania/DAOMod/DAOMod/DAOMod Ontology/myenv/lib/python3.12/site-packages/owlready2/pellet/xml-apis-1.4.01.jar:/home/sowelo/Scrivania/DAOMod/DAOMod/DAOMod Ontology/myenv/lib/python3.12/site-packages/owlready2/pellet/commons-codec-1.6.jar:/home/sowelo/Scr


The ontology is consistent!
Ontology instantiated and saved as 'Circles_Ubi_Instantiated.owl'.

Checking ontology consistency...

The ontology is consistent!

Analyzing ontology...
Number of instances: 18

Instance: CirclesDAO
  Data properties assigned: 2
    - DAO_name: ['Circles DAO']
    - mission_statement: ['Manage governance of Circles group, token economy, and attest user contributions.']
  Object properties assigned: 0

Instance: GroupMember
  Data properties assigned: 1
    - role_description: ['Users depositing Circles tokens to join.']
  Object properties assigned: 0

Instance: ActiveMember
  Data properties assigned: 1
    - role_description: ['Members contributing to the group governance.']
  Object properties assigned: 0

Instance: TreasuryManager
  Data properties assigned: 1
    - role_description: ['Manages group funds.']
  Object properties assigned: 0

Instance: GeneralAssembly
  Data properties assigned: 1
    - n_agents: [100]
  Object properties assigned: 0

Ins

* Owlready2 * Pellet took 0.9437785148620605 seconds
* Owlready * Reparenting DAOMod_Ontology.ServiceProvision: {DAOMod_Ontology.Governance_Area} => {eSourcing_Ontology.Role_Delegation, DAOMod_Ontology.Governance_Area, eSourcing_Ontology.Privilege, eSourcing_Ontology.Capability, eSourcing_Ontology.Appointment, eSourcing_Ontology.Connection_Relationship, eSourcing_Ontology.Correlation, eSourcing_Ontology.Hierarchy_Relationship, eSourcing_Ontology.Sourcing_Sphere, eSourcing_Ontology.Organizational_Unit, eSourcing_Ontology.Power_Delegation, eSourcing_Ontology.Containment_Type, eSourcing_Ontology.Space, eSourcing_Ontology.Power, eSourcing_Ontology.Consideration, eSourcing_Ontology.Role}
* Owlready * Reparenting DAOMod_Ontology.TreasuryManagement: {DAOMod_Ontology.Governance_Area} => {eSourcing_Ontology.Role_Delegation, DAOMod_Ontology.Governance_Area, eSourcing_Ontology.Privilege, eSourcing_Ontology.Capability, eSourcing_Ontology.Appointment, eSourcing_Ontology.Connection_Relationship, eSo